In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re  # regex para validar si existe subfijos

def shirley_correction(raman_shift, intensity):
    """
    Aplica la corrección de Shirley para un espectro.
    
    Parameters:
        raman_shift (array-like): Valores del eje X (Raman Shift).
        intensity (array-like): Intensidades del espectro (eje Y).
        
    Returns:
        corrected_intensity (array-like): Intensidades corregidas.
    """
    if len(raman_shift) != len(intensity):
        raise ValueError("La longitud de 'Ramanshift' y la intensidad no coincide.")
    
    corrected_intensity = intensity.copy()
    start = corrected_intensity[0]
    end = corrected_intensity[-1]
    
    for _ in range(100):  # Máximo 100 iteraciones
        background = start + (end - start) * np.cumsum(corrected_intensity) / np.sum(corrected_intensity)
        corrected_intensity = intensity - background
        corrected_intensity[corrected_intensity < 0] = 0  # Evitar valores negativos
    
    return corrected_intensity

# Leer el archivo CSV
df = pd.read_csv('limpio.csv', delimiter=',')

# Eliminar sufijos numéricos de los encabezados
if any(re.search(r'\.\d+$', col) for col in df.columns):
    df.columns = [re.sub(r'\.\d+$', '', col) for col in df.columns]
    print("Se eliminaron los sufijos numéricos de los encabezados.")

# Muestra las primeras filas del DataFrame para verificar
print(df.head())

# Validar las dimensiones de las columnas antes de aplicar la corrección
raman_shift = df['Ramanshift'].values
corrected_spectra = {}

for col in df.columns[1:]:
    intensity = df[col].values
    # Validar longitud y ajustar si es necesario
    if len(raman_shift) == len(intensity):
        corrected_spectra[col] = shirley_correction(raman_shift, intensity)
    elif len(raman_shift) > len(intensity):
        # Alinear con raman_shift rellenando con ceros
        intensity = np.pad(intensity, (0, len(raman_shift) - len(intensity)), constant_values=0)
        corrected_spectra[col] = shirley_correction(raman_shift, intensity)
    else:
        print(f"Advertencia: La columna '{col}' tiene más datos que 'Ramanshift'. Será recortada.")
        intensity = intensity[:len(raman_shift)]
        corrected_spectra[col] = shirley_correction(raman_shift, intensity)

# Crear un DataFrame con los datos corregidos
df_corrected = pd.DataFrame(corrected_spectra)
df_corrected.insert(0, 'Ramanshift', raman_shift)

# Identificar los tipos únicos de valores en los encabezados
unique_types = set(col for col in df_corrected.columns if col != "Ramanshift")

# Colores para cada tipo
colors = plt.cm.tab20.colors  # Una paleta de colores suficientemente grande
color_map = {unique: colors[i % len(colors)] for i, unique in enumerate(unique_types)}

# Graficar cada tipo una sola vez en la leyenda
plt.figure(figsize=(14, 10))

for unique_type in unique_types:
    # Filtrar las columnas correspondientes al tipo actual
    columns = [col for col in df_corrected.columns if col.startswith(unique_type)]
    
    # Graficar todas las columnas del tipo actual
    for col in columns:
        plt.plot(df_corrected['Ramanshift'], df_corrected[col], color=color_map[unique_type], alpha=0.6)
    
    # Agregar una entrada en la leyenda solo para el tipo (una vez)
    plt.plot([], [], label=unique_type, color=color_map[unique_type])  # Dummy plot for legend

# Etiquetas y leyendas
plt.title("Espectros Raman Corregidos por Shirley", fontsize=16)
plt.xlabel("Raman Shift (cm⁻¹)", fontsize=14)
plt.ylabel("Intensidad Corregida", fontsize=14)
plt.legend(title="Tipos", fontsize=12, loc='upper right', frameon=False)
plt.grid(True)

# Mostrar la gráfica
plt.show()

Se eliminaron los sufijos numéricos de los encabezados.
   Ramanshift  collagen  collagen  collagen  collagen  collagen  collagen  \
0     1801.26     0.117     0.123     0.098     0.097     0.115     0.129   
1     1797.41     0.118     0.124     0.099     0.098     0.116     0.130   
2     1793.55     0.119     0.124     0.100     0.098     0.117     0.131   
3     1789.69     0.118     0.122     0.099     0.097     0.117     0.131   
4     1785.84     0.118     0.121     0.099     0.096     0.116     0.130   

   collagen  collagen  collagen  ...    DNA    DNA    DNA    DNA    DNA  \
0     0.130     0.144     0.129  ...  0.154  0.150  0.154  0.164  0.157   
1     0.131     0.145     0.129  ...  0.154  0.152  0.155  0.164  0.158   
2     0.132     0.145     0.130  ...  0.155  0.153  0.156  0.165  0.160   
3     0.132     0.146     0.131  ...  0.155  0.154  0.157  0.165  0.160   
4     0.131     0.146     0.131  ...  0.155  0.155  0.157  0.166  0.160   

     DNA    DNA    DNA    DNA 

ValueError: operands could not be broadcast together with shapes (195,) (45630,) 